In [2]:
import polars as pl
from polars import col, lit
import polars.selectors as cs
import duckdb as db
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
def sql(query, show=True):
    result = db.sql(query)
    return result

In [4]:
sql(
    """
    select
        *

    from read_parquet('../data/03_preprocessed/registry.parquet')
    """
).to_view("registry")

┌──────────┬────────────────────┬───────────┐
│ match_id │    person_name     │ person_id │
│ varchar  │      varchar       │  varchar  │
├──────────┼────────────────────┼───────────┤
│ 1000851  │ AC Voges           │ 864c199e  │
│ 1000851  │ AJ Pycroft         │ 685d3f80  │
│ 1000851  │ Aleem Dar          │ f02778d1  │
│ 1000851  │ D Elgar            │ 33085ffb  │
│ 1000851  │ DA Warner          │ dcce6f09  │
│ 1000851  │ DW Steyn           │ 8fd1a8f5  │
│ 1000851  │ F du Plessis       │ 3355b542  │
│ 1000851  │ HM Amla            │ e798611a  │
│ 1000851  │ JP Duminy          │ 2e8994e7  │
│ 1000851  │ JR Hazlewood       │ 03806cf8  │
│    ·     │      ·             │    ·      │
│    ·     │      ·             │    ·      │
│    ·     │      ·             │    ·      │
│ 1063066  │ Arafat Sunny       │ 1f7adc8a  │
│ 1063066  │ Imrul Kayes        │ 5fbe14fc  │
│ 1063066  │ LA Dawson          │ 4a461c24  │
│ 1063066  │ Liton Das          │ 0404d43c  │
│ 1063066  │ MN Samuels         │ 

In [5]:
sql(
    """
    select
        *
    from read_parquet('../data/04_processed/cricket_dataset.parquet')
    """
).to_view("t1")

┌─────────────┬──────────┬────────────┬─────────────┬──────────────┬──────────────┬──────────┬────────────────┬───────────────┬───────────────┬──────────────┬──────────┬───────┬───────┬─────────┬─────────────┬────────────┬────────────┬───────┬───────┬─────────────┬───────────┬───────────┬────────────────┬───────────────┬─────────┬──────────────────┬────────────────┬────────────┬─────────┬────────────────┬──────────┬─────────┬────────────┬──────────────┬─────────────────┬───────────┬──────────────┬───────────────┬──────────────┬──────────────────────────────────────────────┬───────────────┐
│ delivery_id │ match_id │ innings_id │   innings   │ batting_team │ bowling_team │ declared │     bowler     │    batter     │  non_striker  │  player_out  │ delivery │ over  │ ball  │ wickets │ batter_runs │ extra_runs │ total_runs │  six  │ four  │ wicket_type │ batter_id │ bowler_id │ non_striker_id │ player_out_id │  city   │ match_start_date │ match_end_date │ match_type │ gender  │    umpire_1

In [6]:
sql(
    """
    select
        * exclude(batter_id, bowler_id, non_striker_id, player_out_id),
        hash(batting_team) as batting_team_hash,
        hash(bowling_team) as bowling_team_hash,
        hash(coalesce(batter_id, batter)) as batter_hash,
        hash(coalesce(bowler_id, bowler)) as bowler_hash,
        hash(coalesce(non_striker_id, non_striker)) as non_striker_hash,
        hash(coalesce(player_out_id, player_out)) as player_out_hash,
    from t1
    """
).to_view("t2")

┌─────────────┬──────────┬────────────┬─────────────┬──────────────┬──────────────┬──────────┬────────────────┬───────────────┬───────────────┬──────────────┬──────────┬───────┬───────┬─────────┬─────────────┬────────────┬────────────┬───────┬───────┬─────────────┬─────────┬──────────────────┬────────────────┬────────────┬─────────┬────────────────┬──────────┬─────────┬────────────┬──────────────┬─────────────────┬───────────┬──────────────┬───────────────┬──────────────┬──────────────────────────────────────────────┬───────────────┬──────────────────────┬──────────────────────┬──────────────────────┬──────────────────────┬──────────────────────┬──────────────────────┐
│ delivery_id │ match_id │ innings_id │   innings   │ batting_team │ bowling_team │ declared │     bowler     │    batter     │  non_striker  │  player_out  │ delivery │ over  │ ball  │ wickets │ batter_runs │ extra_runs │ total_runs │  six  │ four  │ wicket_type │  city   │ match_start_date │ match_end_date │ match_type

In [132]:
hash_dims = ["batting_team_hash", "bowling_team_hash", "batter_hash", "bowler_hash", "non_striker_hash", "player_out_hash"] 

for dim in hash_dims:
    sql(
        f"""
        with cte_1 as (select distinct {dim.split("_hash")[0]}, {dim} from t2)
        select {dim.split("_hash")[0]}, {dim}, row_number() over(order by {dim}) as {dim.split("_hash")[0]}_sk from cte_1
        """
    ).to_view(f"dim_{dim.replace("_hash", "")}")

In [137]:
sql("select * from dim_batting_team limit 1")

┌──────────────┬───────────────────┬─────────────────┐
│ batting_team │ batting_team_hash │ batting_team_sk │
│   varchar    │      uint64       │      int64      │
├──────────────┼───────────────────┼─────────────────┤
│ Essex        │ 62947260050851832 │               1 │
└──────────────┴───────────────────┴─────────────────┘

In [145]:
sql(
    """
    select
        * exclude(
            batter_hash, bowler_hash, batting_team_hash, bowling_team_hash, non_striker_hash, player_out_hash
        )

    from 
        t2 t
        left join (select batter_hash, batter_sk from dim_batter) DimBat
            on DimBat.batter_hash = t.batter_hash
        left join (select bowler_hash, bowler_sk from dim_bowler) DimBowl
            on DimBowl.bowler_hash = t.bowler_hash
        left join (select batting_team_hash, batting_team_sk from dim_batting_team) DimBatting_team
            on DimBatting_team.batting_team_hash = t.batting_team_hash
        left join (select bowling_team_hash, bowling_team_sk from dim_bowling_team) DimBowling_team
            on DimBowling_team.bowling_team_hash = t.bowling_team_hash
        left join (select non_striker_hash, non_striker_sk from dim_non_striker) DimNon_striker
            on DimNon_striker.non_striker_hash = t.non_striker_hash
        left join (select player_out_hash, player_out_sk from dim_player_out) DimPlayer_out
            on DimPlayer_out.player_out_hash = t.player_out_hash
    """
)

┌─────────────┬──────────┬────────────┬─────────────┬───────────────┬─────────────────┬──────────┬──────────────┬───────────────┬──────────────┬───────────────┬──────────┬───────┬───────┬─────────┬─────────────┬────────────┬────────────┬───────┬───────┬───────────────────┬─────────┬──────────────────┬────────────────┬────────────┬─────────┬─────────────────┬──────────┬─────────┬────────────┬─────────────────┬─────────────────┬───────────────┬─────────────────┬───────────────┬───────────────┬──────────────────────────┬───────────────────────┬───────────┬───────────┬─────────────────┬─────────────────┬────────────────┬───────────────┐
│ delivery_id │ match_id │ innings_id │   innings   │ batting_team  │  bowling_team   │ declared │    bowler    │    batter     │ non_striker  │  player_out   │ delivery │ over  │ ball  │ wickets │ batter_runs │ extra_runs │ total_runs │  six  │ four  │    wicket_type    │  city   │ match_start_date │ match_end_date │ match_type │ gender  │    umpire_1     

In [52]:
db.sql(
    """
    with tt1 as (
        select
            match_type,
            league,
            match_start_date as date,
            bowling_team,
            innings_id,
            count(delivery_id) as balls,
            sum(batter_runs) as runs,
            sum(six) as six,
            sum(four) as four,
            case when winner = batting_team then 'win' else 'loss' end as result
        from t1
        where
            batter = 'Rishabh Pant'
        group by all
    )

    select
        match_type,
        league,
        result,
        count(*)
    from tt1
    group by all
    order by match_type, league, result
    """
)

┌────────────┬─────────────────────────────┬─────────┬──────────────┐
│ match_type │           league            │ result  │ count_star() │
│  varchar   │           varchar           │ varchar │    int64     │
├────────────┼─────────────────────────────┼─────────┼──────────────┤
│ ODI        │ International               │ loss    │           16 │
│ ODI        │ International               │ win     │           11 │
│ T20        │ Indian Premier League (IPL) │ loss    │           64 │
│ T20        │ Indian Premier League (IPL) │ win     │           57 │
│ T20        │ International               │ loss    │           24 │
│ T20        │ International               │ win     │           42 │
│ Test       │ International               │ loss    │           32 │
│ Test       │ International               │ win     │           34 │
└────────────┴─────────────────────────────┴─────────┴──────────────┘